# 結業測驗與認證：基於 RAG 的對談系統

## 背景說明：

作為 AI 工程師職位的候選人，請建立一個基於檢索輔助生成（Retrieval-Augmented Generation，RAG）的對談系統。此系統能根據提供的文字及表格來回答問題。以下提供了一個來自城市交通分析數據集的範例表格，您的系統必須能準確地從中檢索資訊並生成回答。

## 問題：

建立一個 RAG 對話系統，能夠處理以下表格資訊，並能準確回答下列問題：
- 「請問台北市大安區與中正區的自行車占比率為多少？」

## 該系統應展示以下能力：

1. 能夠處理文件中的文字與表格內容。
2. 基於內容準確提取資訊並生成自然語言的回答。

## 交付項目：

- 一個展示您所建系統的 Jupyter Notebook。
- 一個展示設計過程與架構的簡報，包括：
  - 系統架構
  - 設計選擇
- 您如何解決文件和圖像數據的 RAG 整合問題。

## 各運具類別市佔率

![各運具類別市佔率](test.jpg)

# 實作 - 處理文件中的文字與表格內容

## jpg 轉存成 json 檔案

### 載入類別

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
import base64
import re
import os
from dotenv import load_dotenv, find_dotenv

### 讀取 env

In [ ]:
_ = load_dotenv(find_dotenv())

wits_gpt_endpoint = os.getenv("WITS_GPT_ENDPOINT")
wits_gpt_apikey = os.getenv("WITS_GPT_API_KEY")
wits_gpt_apiversion = os.getenv("WITS_GPT_API_VERSION")
wits_gpt_model = os.getenv("WITS_GPT_MODEL")

### 定義 Function

#### 將 image 轉成 base64

In [ ]:
def encode_image_to_base64(image_path):
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")
    except Exception as e:
        print(f"Error encoding image: {e}")
        return None

#### 建立 image message

In [ ]:
def create_image_message(base64_image):
    return {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
        }
    }

#### 從 message 抓出 json

In [ ]:
def message_to_json(json_content):
    pattern = r'```json\s*\n(.*?)\n\s*```'
    match = re.search(pattern, json_content, re.DOTALL)
    if match:
        json_content = match.group(1)
        return json_content
    else:
        print("未找到 json block")

#### 將 image 轉成 json 並存檔

In [ ]:
def image_save_to_json(image_path):
    llm = AzureChatOpenAI(
        azure_endpoint=wits_gpt_endpoint,
        openai_api_key=wits_gpt_apikey,
        deployment_name=wits_gpt_model,
        openai_api_version=wits_gpt_apiversion,
        temperature=0
    )
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at {image_path}.")
        return
    base64_image = encode_image_to_base64(image_path)
    if not base64_image:
        print("Error: Failed to encode image")
        return
    image_message = create_image_message(base64_image)
    messages = [
        HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": """請幫我解析這張圖片中的表格，並轉換為 json 格式。過濾掉 ```\s*?\(\d+?\)```。
                    JSON 格式參考：
                    ```
                    [
                        {
                            "District": "大安區",
                            "綠運輸": 72.9,
                            "公共運具": 51.3,
                            "非機動運具": 21.6,
                            "步行": 17.7,
                            "自行車": 3.8,
                            "私人機動運具": 27.1,
                            "最常公共運具使用率": 62.1
                        },
                        ...
                    ]
                    ```

                    輸出放到 json 區塊中：
                    ```json
                    {json_content}
                    ```
                    """
                },
                image_message
            ]
        )
    ]

    try:
        chain = llm | StrOutputParser()
        print("正在將圖片傳送至 GPT...")
        response = chain.invoke(messages)
        json_table = message_to_json(response)
        print("\n轉換結果：")
        print(json_table)
        output_file = "output_table.json"
        with open(output_file, "w", encoding="utf-8") as file:
            file.write(json_table)
        print(f"JSON 表格已保存至 {output_file}")
    except Exception as e:
        print(f"處理過程中發生錯誤: {e}")

### 執行轉換

In [ ]:
image_save_to_json("test.jpg")

## 將 json 檔案存入 VectorStore

### 讀取 json 檔案
https://python.langchain.com/docs/integrations/document_loaders/json/
https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.in_memory.InMemoryVectorStore.html#langchain_core.vectorstores.in_memory.InMemoryVectorStore

#### 安裝 Library

In [ ]:
! pip install langchain_community jq langchain-openai langchain-core

#### 載入類別

In [ ]:
import os, json
from dotenv import load_dotenv, find_dotenv
from langchain_community.document_loaders import JSONLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

#### 讀取 env

In [ ]:
_ = load_dotenv(find_dotenv())

wits_gpt_endpoint = os.getenv("WITS_GPT_ENDPOINT")
wits_gpt_apikey = os.getenv("WITS_GPT_API_KEY")
wits_gpt_apiversion = os.getenv("WITS_GPT_API_VERSION")
wits_gpt_model = os.getenv("WITS_GPT_MODEL")

wits_embedding_endpoint = os.getenv("WITS_EMBEDDING_ENDPOINT")
wits_embedding_apikey = os.getenv("WITS_EMBEDDING_API_KEY")
wits_embedding_apiversion = os.getenv("WITS_EMBEDDING_API_VERSION")
wits_embedding_model = os.getenv("WITS_EMBEDDING_MODEL")

#### 定義 function

##### 初始化 JSONLoader

In [ ]:
def init_jsonloader(json_path):
    def metadata_func(record: dict, metadata: dict) -> dict:
        metadata = record
        return metadata
        
    return JSONLoader(
        file_path=json_path,
        jq_schema='.[]',
        content_key='.District',
        is_content_key_jq_parsable=True,
        text_content=False,
        metadata_func=metadata_func
    )

##### 初始化 Embeddings

In [ ]:
def init_embeddings():
    return AzureOpenAIEmbeddings(
        azure_endpoint=wits_embedding_endpoint,
        azure_deployment=wits_embedding_model,
        openai_api_version=wits_embedding_apiversion,
        api_key=wits_embedding_apikey
    )

##### 初始化 InMemoryVectorStore

In [ ]:
def init_vectorstore(embeddings):
    return InMemoryVectorStore(embeddings)

#### 執行載入 JSON 並存入 VectorStore

In [ ]:
loader = init_jsonloader('output_table.json')
documents = loader.load()
embeddings = init_embeddings()
vectorstore = init_vectorstore(embeddings)
vectorstore.add_documents(documents)

# Chat Bot

step 1 .   圖轉文 -> 文轉向量 -> 存進向量資料庫  
step 2 . user 打的問題 -> 轉向量 -> 向量資料庫比對抓出最接近的資料 -> 塞prompt 給語言模型 -> 回應給user

## 安裝 Library

In [ ]:
! pip install langchain-core langgraph>0.2.27 langchain-openai

## 載入類別

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage, trim_messages, filter_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.prebuilt import create_react_agent

## 讀取 env

In [2]:
_ = load_dotenv(find_dotenv())

wits_gpt_endpoint = os.getenv("WITS_GPT_ENDPOINT")
wits_gpt_apikey = os.getenv("WITS_GPT_API_KEY")
wits_gpt_apiversion = os.getenv("WITS_GPT_API_VERSION")
wits_gpt_model = os.getenv("WITS_GPT_MODEL")

wits_embedding_endpoint = os.getenv("WITS_EMBEDDING_ENDPOINT")
wits_embedding_apikey = os.getenv("WITS_EMBEDDING_API_KEY")
wits_embedding_apiversion = os.getenv("WITS_EMBEDDING_API_VERSION")
wits_embedding_model = os.getenv("WITS_EMBEDDING_MODEL")

## 定義 function

### 初始化 LLM

In [3]:
def init_llm():
    return AzureChatOpenAI(
        azure_endpoint=wits_gpt_endpoint,
        openai_api_key=wits_gpt_apikey,
        api_version=wits_gpt_apiversion,
        azure_deployment=wits_gpt_model
    )

### 初始化 StateGraph (CompiledStateGraph)

In [41]:
import pprint

def init_state_graph(llm, prompt_template, tools):
    def _call_model(state: MessagesState):
        _trimmed_messages = trim_messages(
            state["messages"],
            strategy="last",
            token_counter=len,
            max_tokens=5,
            start_on="human",
            end_on=("human", "tool"),
            include_system=True,
        )
        _agent_executor = create_react_agent(llm, tools)
        _chain = prompt_template | _agent_executor
        _response = _chain.invoke({"messages": _trimmed_messages})
        return {"messages": _response['messages']}
    _graph = StateGraph(state_schema=MessagesState)
    _graph.add_edge(START, "model")
    _graph.add_node("model", _call_model)
    _memory = MemorySaver()
    return _graph.compile(checkpointer=_memory)

## 開始聊天  
你是負責問答任務的助手。使用以下檢索到的 context 來回答問題。如果你不知道答案，就說你不知道。最多使用三個句子並保持答案簡潔。  
Question: {question}  
Context: {context}  
Answer:

In [13]:
from tempfile import TemporaryDirectory

from langchain_community.agent_toolkits import FileManagementToolkit

# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()

In [14]:
toolkit = FileManagementToolkit(
    # root_dir=str(working_directory.name)
)  # If you don't provide a root_dir, operations will default to the current working directory
tools = toolkit.get_tools()

In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage("你是話少的助手。最多使用三個句子回答。"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [42]:
llm = init_llm()
llm = llm.bind_tools(tools)
graph = init_state_graph(llm, prompt_template, tools)
config = {"configurable": {"thread_id": "test_thread"}}

In [43]:
from datetime import datetime
# query = "請問台北市大安區與中正區的自行車市占率為多少？"
query = f'建立一個 "text.txt" 檔案，並寫入 "{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}: 寫入測試"。'

input_messages = [HumanMessage(query)]
output = graph.invoke({"messages": input_messages}, config)

output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

檔案 "text.txt" 已建立，並成功寫入內容。
